In [ ]:
🎙️ Assistente de Voz com OpenRouter — Notebook Colab

In [2]:
#Instalar dependencias

!pip install -q openai-whisper requests gTTS pydub


In [4]:
#Imports e config

import os
import requests
import whisper
from gtts import gTTS
from IPython.display import Audio, display

In [5]:
# Chamando ia do openrputer

OPENROUTER_API_KEY = "COLE SUA CHAVE API KEY AQUI" # ⬅️ INSIRA SUA CHAVE DA OPENROUTER AQUI
# Para maior segurança, considere usar o recurso 'Secrets' do Google Colab para armazenar suas chaves de API.

MODEL = "openrouter/auto" # Define o modelo a ser usado. Você pode alterar para outro modelo suportado pelo OpenRouter.
transcription = "Olá, como você está?" # Placeholder para a transcrição. Isso será preenchido com a transcrição do áudio.

url = "https://openrouter.ai/api/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "model": MODEL,
    "messages": [
        {"role": "user", "content": transcription}
    ]
}

response = requests.post(url, headers=headers, json=payload)

# verificação simples de erro
if response.status_code != 200:
    print(response.text)
else:
    data = response.json()
    assistant_text = data["choices"][0]["message"]["content"]
    print("🤖 Resposta:")
    print(assistant_text)

🤖 Resposta:
Olá! Estou bem, obrigado. E você? Como posso ajudar hoje?


In [6]:
# Gravação com visualizador de áudio (waveform)

from IPython.display import Javascript, display
from google.colab import output
import base64

JS_CODE = """
async function recordWithVisualizer() {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });

  const audioContext = new AudioContext();
  const source = audioContext.createMediaStreamSource(stream);
  const analyser = audioContext.createAnalyser();
  source.connect(analyser);

  analyser.fftSize = 2048;
  const bufferLength = analyser.fftSize;
  const dataArray = new Uint8Array(bufferLength);

  const canvas = document.createElement('canvas');
  canvas.width = 600;
  canvas.height = 200;
  document.body.appendChild(canvas);
  const canvasCtx = canvas.getContext('2d');

  function draw() {
    requestAnimationFrame(draw);

    analyser.getByteTimeDomainData(dataArray);

    canvasCtx.fillStyle = 'black';
    canvasCtx.fillRect(0, 0, canvas.width, canvas.height);

    canvasCtx.lineWidth = 2;
    canvasCtx.strokeStyle = 'lime';
    canvasCtx.beginPath();

    const sliceWidth = canvas.width * 1.0 / bufferLength;
    let x = 0;

    for(let i = 0; i < bufferLength; i++) {
      const v = dataArray[i] / 128.0;
      const y = v * canvas.height / 2;

      if(i === 0) {
        canvasCtx.moveTo(x, y);
      } else {
        canvasCtx.lineTo(x, y);
      }
      x += sliceWidth;
    }

    canvasCtx.lineTo(canvas.width, canvas.height/2);
    canvasCtx.stroke();
  }

  draw();

  const recorder = new MediaRecorder(stream);
  let chunks = [];

  recorder.ondataavailable = e => chunks.push(e.data);
  recorder.start();

  await new Promise(resolve => {
    const btn = document.createElement("button");
    btn.textContent = "⏹️ Parar gravação";
    document.body.appendChild(btn);
    btn.onclick = () => {
      recorder.stop();
      resolve();
      btn.remove();
    };
  });

  await new Promise(resolve => recorder.onstop = resolve);

  const blob = new Blob(chunks);
  const arrayBuffer = await blob.arrayBuffer();
  const base64Audio = btoa(
    new Uint8Array(arrayBuffer)
      .reduce((data, byte) => data + String.fromCharCode(byte), '')
  );

  return base64Audio;
}

recordWithVisualizer();
"""

audio_base64 = output.eval_js(JS_CODE)
audio_bytes = base64.b64decode(audio_base64)

with open("audio.wav", "wb") as f:
    f.write(audio_bytes)

audio_file = "audio.wav"
print("🎤 Áudio gravado com visualização:", audio_file)

🎤 Áudio gravado com visualização: audio.wav


In [7]:

# Transcrição

model = whisper.load_model("base")
result = model.transcribe(audio_file, language="pt")
transcription = result["text"]

print("📝 Transcrição:")
print(transcription)


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


📝 Transcrição:
 Olá, me fale sobre tecnologia ciência de dados de inteligência artificial de forma curta e rápida.


In [8]:
# Chamada ao OpenRouter
url = "https://openrouter.ai/api/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "model": MODEL,
    "messages": [
        {"role": "user", "content": transcription}
    ]
}

response = requests.post(url, headers=headers, json=payload)
data = response.json()

assistant_text = data["choices"][0]["message"]["content"]

print("🤖 Resposta:")
print(assistant_text)

🤖 Resposta:
Resumo rápido:

- IA e Ciência de Dados: IA permite máquinas aprenderem/performarem tarefas; Ciência de Dados usa dados, estatística e ML para extrair insights e tomar decisões.

- Tecnologias-chave: Python (e bibliotecas), SQL, R; frameworks/motores como Scikit-learn, TensorFlow, PyTorch; nuvem e notebooks.

- Fluxo típico: coletar/limpar dados → engenharia de features → treinar e avaliar modelos → implementar e monitorar (MLOps).

- Áreas e aplicações: aprendizado de máquina (supervisionado/não supervisionado), visão computacional, NLP; aplicações em recomendações, detecção de fraudes, diagnósticos, previsões de demanda.

- Tendências rápidas: modelos de base grandes, IA ética e governança, eficiência/edge AI, automação de pipelines; começo rápido com Python e datasets públicos.

Quer que eu aprofunde algum tópico específico?


In [9]:
# Converter resposta em voz

tts = gTTS(
    text=assistant_text[:4000],
    lang="pt",
    tld="com.br",   # voz brasileira
    slow=False
)
tts.save("resposta.mp3")

from IPython.display import Audio
Audio("resposta.mp3")
